In [ ]:
import os
import requests as re
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
from io import BytesIO
from tqdm.auto import tqdm
import time
import glob
from datetime import datetime
import io
import zipfile

import xml.etree.ElementTree as ET

import seaborn as sns
import matplotlib.pyplot as plt

try:
    from pandas import json_normalize
except ImportError:
    from pandas.io.json import json_normalize

from crawling.crawler import *
from preprocessing.preprocessing_dart import DartFinstate, calculate_annualized
from crawling.crawling_dart import CrawlingDart

from database.access import AccessDataBase
from preprocessing.finance_ratio import GetRatio

## connect database
db_dart = AccessDataBase('root', 'jys1013011!', 'dart')
db_dart_bak = AccessDataBase('root', 'jys1013011!', 'dart_bak')

---
### Kospi

---
#### Crawling: dart_finstatements

In [ ]:
# stock codes
stocks_df = db_dart.get_tbl('dart_stocks')
stock_codes = stocks_df.stock_code.unique()

In [ ]:
# Test: kospi 200
stock_codes = stock_codes[:200]

In [ ]:
# get finstatements to dart 
crw = CrawlingDart()
crw.quarters = ['Q202211012']

fins, error, error_none = [], [], []
for stock_code in tqdm(stock_codes):
    _fins, _error, _error_none = crw.get_finstates(stock_code)
    fins += _fins
    error += _error
    error_none += _error_none
print(len(stock_codes), len(error), len(error_none))

# re-get error: None type
fins_re_0, error_re_0 = [], []
for codeq in tqdm(error):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_0.append(fin)
    else:
        error_re_0.append([code, q])
# print(len(fins_re_0), len(error_re_0))

# re-get error: The others
fins_re_1, error_none_re_1 = [], []
for codeq in tqdm(error_none):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_1.append(fin)
    else:
        error_none_re_1.append([code, q])
print(len(fins_re_1), len(error_none_re_1))

# re-get error 2: None type
fins_re_2, error_re_2 = [], []
for codeq in tqdm(error_re_0):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_2.append(fin)
    else:
        error_re_2.append([code, q])
print(len(fins_re_2), len(error_re_2))
        
if len(error_re_0) == len(error_re_2) and len(fins_re_2) == 0:
    print('Complete!')

---
#### Upload: dart_finstatements

In [ ]:
# concat & dedup
_fins_ = fins + fins_re_0 + fins_re_1
df_concat_kospi = pd.concat(_fins_)

columns = ['rcept_no', 'reprt_code', 'bsns_year', 'corp_code', 'sj_div', 'sj_nm', 'fs_div', 'stock_code', 'account_id', 'account_nm', 'thstrm_nm', 'thstrm_amount']
df_dedup_kospi = df_concat_kospi.drop_duplicates(subset=columns, keep='first', ignore_index=True)

fins_df = df_dedup_kospi.loc[:, columns]
fins_df.loc[fins_df.thstrm_amount=='', 'thstrm_amount'] = None
fins_df.loc[:, 'thstrm_amount'] = fins_df.thstrm_amount.astype('float64')

In [ ]:
fins_df.info()
fins_df.head()

# print(len(fins_df.stock_code.unique()))

In [ ]:
# upload dart_finstatements
# db_dart_bak.engine_upload(fins_df, 'dart_finstatements', 'append')

---
#### Upload: dart_amounts

In [ ]:
''' Create quarter amount all table '''

# Kospi: All
fins_df = db_dart_bak.get_tbl('dart_finstatements')
dartfins = DartFinstate(fins_df)
amounts_all_df = dartfins.create_amount_quarter()
amounts_all_df.groupby('sj_div').count()

# upload table
# db_dart_bak.create_table(upload_df=amounts_all_df, table_name='dart_amounts_all')

In [ ]:
''' Get confirmed account amount 
    All data (replace) '''

fins_df = pd.DataFrame()
dartfins = DartFinstate(fins_df)
amounts_all_df = db_dart_bak.get_tbl('dart_amounts_all')
accounts_df = db_dart_bak.get_tbl('dart_accounts')
amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)
amounts_df.groupby('account_nm_eng').count()['stock_code']

In [ ]:
''' Get confirmed account amount 
    New data (append) '''

dartfins = DartFinstate(pd.DataFrame())
amounts_all_df = db_dart_bak.get_tbl('dart_amounts_all')

accounts_df = db_dart_bak.get_tbl('dart_accounts')
dart_amounts_df = db_dart.get_tbl('dart_amounts')
confirmed_accounts = dart_amounts_df.account_nm_eng.unique().tolist()

new_accounts = []
for account in accounts_df.account_nm_eng.unique():
    if account in confirmed_accounts:
        pass
    else:
        new_accounts.append(account)
accounts_df = accounts_df[accounts_df.account_nm_eng.isin(new_accounts)].reset_index(drop=True)

if accounts_df.empty:
    print('Dataframe empty')
else:
    amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)
    amounts_df.groupby('account_nm_eng').count()['stock_code']

In [ ]:
''' Caculate 4th quarter net '''

df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)
df_amounts_quarter_db = df_amounts_quarter.drop(columns=['corp_code', 'account_nm'])
df_amounts_quarter_db.info()
df_amounts_quarter_db.tail(5)

In [ ]:
''' Upload table: dart_amounts '''

table = 'dart_amounts'
fields = tuple(df_amounts_quarter_db.columns)
data = df_amounts_quarter_db.values.tolist()
db_dart.insert_many(table_name=table, fields=fields, data=data)

# db_dart.engine_upload(upload_df=df_amounts_quarter_db, table_name=table, if_exists_option='append')

In [ ]:
''' Upload table: dart_accounts '''

table = 'dart_accounts'
fields = tuple(accounts_df.columns)
data = accounts_df.values.tolist()
db_dart.insert_many(table_name=table, fields=fields, data=data)
    
# db_dart.engine_upload(upload_df=accounts_df, table_name=table, if_exists_option='append')

---
#### Django table delete | truncate

In [ ]:
# tables = ['dart_amounts', 'dart_accounts']
tables = ['dart_accounts']
app_name = 'django'

In [ ]:
# Drop

conn, curs = db_dart._connect()

for table in tables:
    query = f'DROP TABLE {table};'
    curs.execute(query)

query = f"DELETE FROM `{app_name}_migrations` WHERE app='{app_name}';"
curs.execute(query)

conn.commit()
curs.close()
conn.close()

In [ ]:
# Truncate

conn, curs = db_dart._connect()

for table in tables:
    query = f'TRUNCATE TABLE {table};'
    curs.execute(query)

conn.commit()
curs.close()
conn.close()

---
#### Dart table backup 

In [ ]:
# fins_df = db_dart.get_tbl('dart_finstatements').drop(columns='id')
# accounts_df = db_btkr.get_tbl('dart_accounts_id')
# accounts_all_df = db_btkr.get_tbl('dart_finstatement_accounts')

In [ ]:
# db_dart_bak.create_table(upload_df=fins_df, table_name='dart_finstatements')
# db_dart_bak.create_table(upload_df=amounts_all_df, table_name='dart_amounts_all') 
# db_dart_bak.create_table(upload_df=accounts_df, table_name='dart_accounts')
# db_dart_bak.create_table(upload_df=accounts_all_df, table_name='dart_accounts_all')

In [ ]:
# backup: `dart_amounts`
tbl = 'dart_amounts'
df = db_dart.get_tbl(tbl)
db_dart_bak.create_table(upload_df=df, table_name=tbl)

In [ ]:
# backup: `dart_accounts`
tbl = 'dart_accounts'
df = db_dart.get_tbl(tbl)
db_dart_bak.create_table(upload_df=df, table_name=tbl)

---
### Kosdaq

In [ ]:
# today 
today_date = datetime.today().strftime('%Y%m%d')

# market
market = "kosdaq"

# 주가정보, 거래대금 및 주식 수 정보
stocks = get_stock_info(market, today_date)
    
# 보통주 종목코드 
common =  get_common_stock_info(market)

# stocks에서 보통주만 추출 
common_stocks = common.loc[:, ['종목코드']].merge(stocks, on='종목코드', how='inner').sort_values(by='시가총액', ascending=False).reset_index(drop=True)

# stock codes
stock_codes = common_stocks.종목코드.unique().tolist()


In [ ]:
# get finstatements to dart 
crw = CrawlingDart()

fins, error, error_none = [], [], []
for stock_code in tqdm(stock_codes):
    _fins, _error, _error_none = crw.get_finstates(stock_code)
    fins += _fins
    error += _error
    error_none += _error_none
print(len(stock_codes), len(error), len(error_none))

# re-get error: None type
fins_re_0, error_re_0 = [], []
for codeq in tqdm(error):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_0.append(fin)
    else:
        error_re_0.append([code, q])
print(len(fins_re_0), len(error_re_0))

# re-get error: The others
fins_re_1, error_none_re_1 = [], []
for codeq in tqdm(error_none):
    code = codeq[0]
    q = codeq[1]
    fin, status = crw.get_finstate(code, q)
    
    if status == 1:
        fins_re_1.append(fin)
    else:
        error_none_re_1.append([code, q])
print(len(fins_re_1), len(error_none_re_1))


# concat & dedup
_fins_ = fins + fins_re_0 + fins_re_1
df_concat_kosdaq = pd.concat(_fins_)
df_dedup_kosdaq = df_concat_kosdaq.drop_duplicates(keep='first')

---
### Test

In [ ]:
''' Test 1: Create Table '''

# conn, curs = db_dart_bak._connect()

# query = '''\
# CREATE TABLE `dart_accounts_test` (\
# `id` bigint AUTO_INCREMENT NOT NULL PRIMARY KEY,\
# `account_nm_eng` varchar(255) NOT NULL,\
# `account_id` varchar(255) NOT NULL,\
# `account_nm_kor` varchar(255) NOT NULL);'''

# curs.execute(query)

# conn.commit()
# curs.close()
# conn.close()

table='dart_accounts_test'
fields=('account_nm_eng', 'account_id', 'account_nm_kor')

for i in range(len(accounts_df)):
    value = tuple(accounts_df.loc[i].values)
    db_dart_bak.insert(table=table, fields=fields, values=value)

In [ ]:
# ''' Test 2: finstatement all '''
# df = db_dart_bak.get_tbl('dart_finstatements')

# code = df.sample(1).stock_code.values[0]
# sj_div = 'BS'
# thstrm_nm = 'Y202111011'

Downloads = '/Users/yeonseosla/Downloads'
df.loc[(df.stock_code==code) & (df.sj_div=='BS') & (df.thstrm_nm=='Y202111011')].to_csv('/Users/yeonseosla/Downloads/bs.csv', index=False)

In [ ]:
''' Test 3: update_amounts '''

def update_amounts(accounts_df, amounts_all_df):
    ''' Get confirmed account amount 
        New data (append) '''

    dart_amounts_df = db_dart.get_tbl('dart_amounts')
    confirmed_accounts = dart_amounts_df.account_nm_eng.unique().tolist()

    new_accounts = []
    for account in accounts_df.account_nm_eng.unique():
        if account in confirmed_accounts:
            pass
        else:
            new_accounts.append(account)
    accounts_df = accounts_df[accounts_df.account_nm_eng.isin(new_accounts)].reset_index(drop=True)

    if accounts_df.empty:
        status = 0
        accounts_new = None
        print('Dataframe empty')
    else:
        status = 1
        accounts_new = accounts_df.account_nm_eng.unique().tolist()
        amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)
        
    if status == 1:
        # Caculate 4th quarter net
        df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
        df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)
        df_amounts_quarter_db = df_amounts_quarter.drop(columns=['corp_code'])

        # Upload table: dart_amounts
        table = 'dart_amounts'
        fields = tuple(df_amounts_quarter_db.columns)
        data = df_amounts_quarter_db.values.tolist()
        db_dart.insert_many(table_name=table, fields=fields, data=data)

    return status, accounts_new

In [ ]:
accounts_df = db_dart_bak.get_tbl('dart_accounts')
dart_amounts_df = db_dart.get_tbl('dart_amounts')
confirmed_accounts = dart_amounts_df.account_nm_eng.unique().tolist()

new_accounts = []
for account in accounts_df.account_nm_eng.unique():
    if account in confirmed_accounts:
        pass
    else:
        new_accounts.append(account)
accounts_df = accounts_df[accounts_df.account_nm_eng.isin(new_accounts)].reset_index(drop=True)
accounts_df

In [ ]:
# accounts_new = accounts_df.account_nm_eng.unique().tolist()
# dartfins = DartFinstate(pd.DataFrame())
# amounts_all_df = db_dart_bak.get_tbl('dart_amounts_all')
# amounts_df = dartfins.get_amounts(amounts_all_df, accounts_df)

# Caculate 4th quarter net
# df_amounts_validitied = amounts_df.loc[amounts_df.validity].reset_index(drop=True)
# df_amounts_quarter = dartfins.calculate_quarter(df_amounts_validitied)
df_amounts_quarter_db = df_amounts_quarter.drop(columns='corp_code')

In [ ]:
df_amounts_quarter_db

---
### ROE

In [ ]:
# df_amounts_quarter = db_dart.get_tbl('dart_amounts')
# quarters = DartFinstate(pd.DataFrame()).quarters_q

# df_amounts_annualized = calculate_annualized(quarters_i=4) # annualized
df_amounts_annualized_octa = calculate_annualized(quarters_i=8) # annualized_octa

In [ ]:
# db_dart_bak.create_table(upload_df=df_amounts_annualized, table_name='dart_annualized')
# db_dart_bak.create_table(upload_df=df_amounts_annualized_octa, table_name='dart_annualized_octa')

In [ ]:
# ROE 계산을 위한 필수 계정과목 금액 존재하는 종목만 추출
# essential_accounts = ['revenue', 'assets', 'equity', 'tax_expense', 'profit_before_tax', 'income', 'owner_equity']
essential_accounts = ['owners_profit', 'owner_equity']

codes = []
for code in df_amounts_annualized_octa.stock_code.unique():
    accounts = df_amounts_annualized_octa.loc[df_amounts_annualized_octa.stock_code==code, 'account_nm_eng'].tolist()
    status = 1
    for account in essential_accounts:
        if account not in accounts:
            status = 0
    
    if status == 1:
        codes.append(code)
df_amounts_essential = df_amounts_annualized_octa.loc[df_amounts_annualized_octa.stock_code.isin(codes)].reset_index(drop=True)
df_amounts_essential = df_amounts_essential[df_amounts_essential.Q202211013.notnull()].reset_index(drop=True)
print('stock_code counts: ', len(df_amounts_essential.stock_code.unique()))

In [ ]:
# def get_roe(df_amounts_annual, stock_code):
#     ''' Get roe (std) '''
    
#     _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
#     accounts = _df_corp.account_nm_eng.unique().tolist()
    
#     # 지배지분귀속 당기순이익
#     owners_profit = _df_corp.loc[_df_corp.account_nm_eng=='owners_profit'].iloc[0, 7:]

#     # 지배주주지분
#     owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]
    
#     # roe
#     ROE = owners_profit / owner_equity

#     roe_corp = pd.DataFrame([owners_profit, owner_equity, ROE]).reset_index(drop=True)

#     roe_corp.loc[:, 'stock_code'] = stock_code
#     i = 0
#     for indicator in ['owners_profit', 'owner_equity', 'ROE']:
#         roe_corp.loc[i, 'ratio'] = indicator
#         i += 1
        
#     return roe_corp

In [ ]:
def get_roe(df_amounts_annual, stock_code):
    ''' Get roe (3 step) 
        ROE = NPM * Asset Turnover * Equity Multiplier

        - NPM (Net Profit Margin, the measure of operating efficiency)
            
            지배지분귀속 당기순이익 / 매출 
            
        - Asset Turnover (the measure of asset use efficiency)
            
            매출 / 자산
            
        - Equity Multiplier (the measure of financial leverage)
            
            자산 / 자기자본
    ''' 
    
    _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
    accounts = _df_corp.account_nm_eng.unique().tolist()
    
    # 지배지분귀속 당기순이익
    owners_profit = _df_corp.loc[_df_corp.account_nm_eng=='owners_profit'].iloc[0, 7:]
    
    # 매출액
    revenue = _df_corp.loc[_df_corp.account_nm_eng=='revenue'].iloc[0, 7:]
    
    # 자산
    assets = _df_corp.loc[_df_corp.account_nm_eng=='assets'].iloc[0, 7:]
    
    # 지배주주지분
    owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]
    
    # NPM
    NPM = owners_profit / revenue
    
    # AT
    AT = revenue / assets
    
    # EM 
    EM = assets / owner_equity
    
    # ROE 3 step
    ROE = NPM * AT * EM

    roe_corp = pd.DataFrame([owners_profit, revenue, assets, owner_equity, NPM, AT, EM, ROE]).reset_index(drop=True)

    roe_corp.loc[:, 'stock_code'] = stock_code
    i = 0
    for indicator in ['owners_profit', 'revenue', 'assets', 'owner_equity', 'NPM', 'AT', 'EM', 'ROE']:
        roe_corp.loc[i, 'ratio'] = indicator
        i += 1
        
    return roe_corp

In [ ]:
# def get_roe(df_amounts_annual, stock_code):
#     ''' Get roe (step 5) '''
    
#     _df_corp = df_amounts_annual.loc[df_amounts_annual.stock_code==stock_code]
#     accounts = _df_corp.account_nm_eng.unique().tolist()
    
#     ## IS ##
#     # 매출액
#     revenue = _df_corp.loc[_df_corp.account_nm_eng=='revenue'].iloc[0, 7:]
#     # 법인세비용
#     tax_expense = _df_corp.loc[_df_corp.account_nm_eng=='tax_expense'].iloc[0, 7:]
#     # 세전계속사업이익
#     profit_before_tax = _df_corp.loc[_df_corp.account_nm_eng=='profit_before_tax'].iloc[0, 7:]
    
#     # 매출원가
#     if 'cost_of_sales' in accounts:
#         cost_of_sales = _df_corp.loc[_df_corp.account_nm_eng=='cost_of_sales'].iloc[0, 7:]
#     else:
#         cost_of_sales = 0
    
#     # 판매비와관리비
#     if 'administrative_expenses' in accounts:
#         administrative_expenses = _df_corp.loc[_df_corp.account_nm_eng=='administrative_expenses'].iloc[0, 7:]
#     else:
#         administrative_expenses = 0
    
#     # 이자비용
#     if 'interest_expense' in accounts:
#         interest_expense = _df_corp.loc[_df_corp.account_nm_eng=='interest_expense'].iloc[0, 7:]
#     else:
#         interest_expense = 0
    
#     # 영업외손익
#     # if 'other_gains' in accounts:
#     #     other_gains = _df_corp.loc[_df_corp.account_nm_eng=='other_gains'].iloc[0, 7:]
#     # else:
#     #     other_gains = 0

#     ## BS
#     assets = _df_corp.loc[_df_corp.account_nm_eng=='assets'].iloc[0, 7:]
#     equity = _df_corp.loc[_df_corp.account_nm_eng=='equity'].iloc[0, 7:]
#     owner_equity = _df_corp.loc[_df_corp.account_nm_eng=='owner_equity'].iloc[0, 7:]


#     '''- OPM (Operting Profit Magin)
#         EBIT / revenue
        
#         EBIT = revenue - cost_of_sales - administrative_expenses + other_gains - other_losses
#             = 당기순이익 + 이자비용 + 법인세비용 - 중단영업이익 + 중단영업손실'''

#     EBIT = revenue - cost_of_sales - administrative_expenses
#     if EBIT.sum() == revenue.sum():
#         EBIT = _df_corp.loc[_df_corp.account_nm_eng=='income'].iloc[0, 7:]
#     else:
#         pass
#     OPM = EBIT / revenue

#     '''- AT (Asset Turnover)
#         매출 / 자산'''
#     AT = revenue / assets

#     '''- IER (Interst Expense Rate)
#         이자비용 / 자산'''
#     IER = interest_expense / assets

#     '''- EM (Equity Multiplier)
#         자산 / 자기자본'''
#     EM = assets / equity      
        
#     '''- TRR (Tax Retention Rate)
#         1 - 법인세율'''
        
#     TRR = 1 - tax_expense / profit_before_tax

#     '''- ROE (Return On Equity)
#         (OPM * AT - IER) * EM * TRR'''
#     ROE = (OPM * AT - IER) * EM * TRR

#     roe_corp = pd.DataFrame([OPM, AT, IER, EM, TRR, ROE, owner_equity]).reset_index(drop=True)

#     roe_corp.loc[:, 'stock_code'] = stock_code
#     i = 0
#     for indicator in ['OPM', 'AT', 'IER', 'EM', 'TRR', 'ROE', 'owner_equity']:
#         roe_corp.loc[i, 'ratio'] = indicator
#         i += 1
        
#     return roe_corp

In [ ]:
stock_codes = df_amounts_essential.stock_code.unique()
df_list, error = [], []
for stock_code in tqdm(stock_codes):
    
    try:
        roe_corp = get_roe(df_amounts_essential, stock_code)
        df_list.append(roe_corp)
    except IndexError:
        # 필수 계정과목이 누락된 경우
        error.append(stock_code)
    
roe_df = pd.concat(df_list).reset_index(drop=True)

---
### RIM

- corp_val = equity + equity * (roe - r) * (w / (1 + r - w))  
- price_sell = (equity + equity * (roe - r) / r) / stock_num (w = 1)  
- price_prop = (equity + equity * (roe - r) * 0.9 / (1 + r - 0.9)) / stock_num (w = 0.9)  
- price_buy = (equity + equity * (roe - r) * 0.8 / (1 + r - 0.8)) / stock_num (w = 0.8)  

In [ ]:
stocks_df = db_dart.get_tbl('dart_stocks')
stock_codes = roe_df.stock_code.unique()
columns = roe_df.columns[:10]
n = len(columns)
values_df = stocks_df.copy()

rate = 0.08
roe_df_ = roe_df[(roe_df.Q202211013>rate) & (roe_df.ratio=='ROE')]
stock_codes = roe_df_.stock_code.unique()

In [ ]:
for stock_code in tqdm(stock_codes):
    roe = roe_df.loc[(roe_df.stock_code==stock_code) & (roe_df.ratio=='ROE'), columns].values[0]
    equity = roe_df.loc[(roe_df.stock_code==stock_code) & (roe_df.ratio=='owner_equity'), columns].values[0]
    shares = stocks_df.loc[stocks_df.stock_code==stock_code, 'shares'].values[0]
    
    # buy price
    weight = 0.8
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_buy = corp_val / shares
    columns_buy = columns + '_buy'
    values_df.loc[values_df.stock_code==stock_code, columns_buy] = price_buy
    
    # sell price
    weight = 1
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_sell = corp_val / shares
    columns_sell = columns + '_sell'
    values_df.loc[values_df.stock_code==stock_code, columns_sell] = price_sell
    
    # prop price
    weight = 0.9
    corp_val = equity + equity * (roe - rate) * (weight / (1 + rate - weight)) 
    price_prop = corp_val / shares
    columns_prop = columns + '_prop'
    values_df.loc[values_df.stock_code==stock_code, columns_prop] = price_prop

In [ ]:
roe_df_

In [ ]:
# values_df.loc[:, 'buy_ratio'] = (values_df.Q202211013_buy - values_df.close) / values_df.Q202211013_buy

min_buy_ratio = 0.5
values_df.loc[values_df.buy_ratio>min_buy_ratio].sort_values('buy_ratio', ascending=False, ignore_index=True)

In [ ]:
values_df.info()

In [ ]:
sns.kdeplot((values_df.Q202211013_buy - values_df.close) / values_df.Q202211013_buy)

In [ ]:
sns.kdeplot(values_df.loc[values_df.Q202211013_buy > 0, 'Q202211013_buy'])

---
### Finanace ratio

In [ ]:
ratios = [
    'impaired_capital', # 자본잠식률 = (자본금 - 지배주주지분) / 자본금
    'bis_ratio', # 자기자본비율 = 자본총계 / 자산총계
    'current_ratio', # 유동비율 = 유동자산 / 유동부채
    'quick_ratio', # 당좌비율 = 당좌자산 / 유동부채
    '', # 수중유동성 = (현금, 예금, 유가증권, 즉시 현금화 가능 자산, 즉시 조달가능 금액) / 월매출(최근 12개월 평균)
    '', # D/E 비율 = 유이자부채 / 자기자본
    'ROE', # ROE = 지배지분귀속 당기순이익 (8분기 연율화) / 자기자본 (전기, 당기 평균)
]

In [ ]:
ratio_expression = {
    'impaired_capital': 'issued_capital -- equity ++ non_controlling_equity / issued_capital', # 자본잠식률 = (자본금 - 지배주주지분) / 자본금
    'bis_ratio': 'equity / assets', # 자기자본비율 = 자본총계 / 자산총계
    'currnet_ratio': 'current_assets / current_liabilities', # 유동비율 = 유동자산 / 유동부채
    # 'quick_ratio': 'quick_assets / current_liabilities', # 당좌비율 = 당좌자산 / 유동부채
    'quick_ratio': 'current_assets -- investories / current_liabilities', # 당좌비율 = (유동자산 - 재고자산) / 유동부채
    'cash_liquidity': 'quick_assets / revenue', # 수중유동성 = (현금, 예금, 유가증권, 즉시 현금화 가능 자산, 즉시 조달가능 금액) / 월매출(최근 12개월 평균)
}

# input data
ratio = 'quick_ratio'
expression = ratio_expression['quick_ratio']

# save
ratio_expression[ratio] = expression

expressions = expression.split(' ')
expressions

In [ ]:
getr = GetRatio() 
ratios, errors = getr.get_ratios(ratio, expressions)

quarters = DartFinstate(pd.DataFrame()).quarters_q
columns = ['stock_code', 'ratio'] + quarters
ratio_df = pd.DataFrame(ratios, columns=columns)
ratio_df.sort_values(by='Q202211013', ignore_index=True)

In [ ]:
data = ratio_df.sort_values(by='Q202211013', ignore_index=True)['Q202211013']
sns.kdeplot(data)

In [ ]:
## Insert data ##

fields = tuple(columns)

db_dart.insert_many(table_name='dart_ratios', fields=fields, data=ratios)

ratios_df = db_dart.get_tbl('dart_ratios')
ratios_df.groupby('ratio').count()

In [ ]:
## Table Backup ##

# ratios_df = db_dart.get_tbl('dart_ratios')
# db_dart_bak.create_table(upload_df=ratios_df, table_name='dart_ratios')

In [ ]:
# ## Delete data ##

# conn, curs = db_dart._connect()
# table = 'dart_ratios'

# # Enter the WHERE condition
# where_condition = "ratio='quick_ratio'"

# query = f'DELETE FROM {table} WHERE {where_condition};'

# curs.execute(query)
# conn.commit()

# curs.close()
# conn.close()

In [ ]:
# # Search amouts by stock code
# getr.amounts_df[getr.amounts_df.stock_code=='006340.KS']

In [ ]:
## Test ##

# ratio_df = pd.DataFrame(columns=columns)
# db_dart_bak.engine_upload(upload_df=ratio_df, table_name='dart_ratios_test', if_exists_option='append')

# db_dart_bak.insert_many(table_name='dart_ratios_test', fields=tuple(columns), data=ratios)
# db_dart_bak.get_tbl('dart_ratios_test')

---
### Stocks data

In [ ]:
# today 
today_date = datetime.today().strftime('%Y%m%d')

# market
market = "kospi"

# 주가정보, 거래대금 및 주식 수 정보
stocks = get_stock_info(market, today_date)
    
# 보통주 종목코드 
common =  get_common_stock_info(market)

# stocks에서 보통주만 추출 
common_stocks = common.loc[:, ['stock_code']].merge(stocks, on='stock_code', how='inner').sort_values(by='market_cap', ascending=False).reset_index(drop=True)
# common_stocks = common_stocks.rename(columns={'종목코드': 'stock_code', '종목명': 'stock_name', '시장구분': 'market', '종가': 'close', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume', '시가총액': 'market_cap', '상장주식수': 'shares'})

# stock codes
stock_codes = common_stocks.stock_code.unique().tolist()

In [ ]:
common_stocks

In [ ]:
# db_dart.engine_upload(upload_df=common_stocks, table_name='dart_stocks', if_exists_option='append')
updated = datetime.today().strftime('%Y-%m-%d')
ranges = range(len(common_stocks))
for idx in tqdm(ranges):
    stock_code = common_stocks.loc[idx, 'stock_code']
    close, open, high, low, volume, amounts, market_cap, shares = common_stocks.iloc[idx, 3:].values

    query = f'\
    UPDATE `dart_stocks` \
    SET close={close}, open={open}, low={low}, volume={volume}, amounts={amounts}, market_cap={market_cap}, shares={shares}, updated="{updated}" \
    WHERE stock_code="{stock_code}"'

    conn, curs = db_dart._connect()
    curs.execute(query)
    conn.commit()
    curs.close()
    conn.close()

In [ ]:
stocks_df = db_dart.get_tbl('dart_stocks')
stocks_df.sort_values('amounts').head(60)

In [ ]:
stocks_df.amounts.sum()